In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split 
from pandas.plotting import scatter_matrix
from sklearn import svm

In [ ]:
datas = pd.read_excel('input/train.xls')

In [ ]:
print(datas.info())

In [ ]:
datas['icerik'][:5]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(datas.loc[:,'icerik'],datas.drop(['id','icerik'],axis=1), random_state=0)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_test.shape

In [ ]:
y_test.shape

In [ ]:
knn=KNeighborsClassifier(n_neighbors=1)

In [ ]:
z=[0,1,2,3,4,5,6,7]
z[0]=datas.loc[:,'Gecikti veya Dağıtıma Çıkmadı']
z[1]=datas.loc[:,'Evde yok notu düşüldü veya Kapıya Getirilmedi']
z[2]=datas.loc[:,'Telefonlara Cevap Verilmedi']
z[3]=datas.loc[:,'İade Süreci']
z[4]=datas.loc[:,'Teslim Alınmadı veya Teslim Edilmedi']
z[5]=datas.loc[:,'Kötü Diyalog Veya Saygısız Tutum']
z[6]=datas.loc[:,'Hasarlı veya Kayıp Paket']
z[7]=datas.loc[:,'Hijyen Kurallarına Uyulmadı']

In [ ]:
sikayetsayisi=len(datas)
yontem=HashingVectorizer

In [ ]:
tks = '[A-Za-z0-9]+(?=\\s+)'


pl = Pipeline([
        ('vec', yontem(token_pattern = tks)),
        ('clf', OneVsRestClassifier(LogisticRegression()))
    ])

pl.fit(X_train,y_train)

In [ ]:
test = datas.fillna("")
predictions = pl.predict_proba(test.icerik)

In [ ]:
toplam=[0,0,0,0,0,0,0,0]
ortalama=[0,0,0,0,0,0,0,0]
for sutun in range (0,len(z)):
    for i in range (0,len(predictions)):
        toplam[sutun]=toplam[sutun]+predictions[i][sutun]
    ortalama[sutun]=toplam[sutun]/len(predictions)

In [ ]:
for i in range (0 , len(predictions)):
    for j in range (0,len(z)):
        if predictions[i][j] < ortalama[j]:
            predictions[i][j] = 0
        else:
            predictions[i][j] = 1

In [ ]:
# Format predictions in DataFrame: prediction_df
prediction_df = pd.DataFrame(columns=y_test.columns, index=test.id, data=predictions)

prediction_df.to_excel('predictions.xls')

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
y_train

In [ ]:
y_test

In [ ]:
clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)